# Подключение библиотек

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

# Задание 1

## Загрузка данных

In [ ]:
url='https://drive.google.com/file/d/1cx0pshptDSVmaWLJCBGS9jIIJ2g-VRgT/view'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)

df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## Цель задания  



- Построить линейную модель.
  - Независимая переменная – цена на недвижимость (`price`)
  - Зависимые переменные:
    - `sqft_living` – жилая площадь
    - `sqft_lot` – площадь участка
    - `sqft_above` – площадь над уровнем земли
- Вычислить оценки коэффициентов и остаточной дисперсии.
- Построить доверительные интервалы для коэффициентов.
- Вычислить коэффициент детерминации $R^2$.
- Проверить гипотезы:
  - Чем больше жилая площадь, тем выше цена.
  - Цена зависит от `sqft_lot`.
  - Гипотеза $H_0$: коэффициенты при `sqft_living` и `sqft_above` равны нулю одновременно против $H_1$ = не $H_0$.

## Модель линейной регрессии



Модель линейной регрессии имеет вид:

$$
y = X\beta + \varepsilon
$$

где:

- $y$ – вектор зависимой переменной (`price`)

- $X$ – матрица независимых переменных, с добавлением столбца единиц для свободного коэффициента (`sqft_living`, `sqft_lot`, `sqft_above`)

- $\beta$ – вектор коэффициентов модели

- $\varepsilon$ – вектор ошибок, ($ε​∼\mathcal N(0,σ^2)$, из условия нормальности и гомоскедастичности)


Вектор оценки параметров $\hat{\beta}$ можно найти методом наименьших квадратов:

$$
\hat{\beta} = (X^T X)^{-1} X^T y
$$



In [ ]:
# Выделение необходимых данных
X = df[['sqft_living', 'sqft_lot', 'sqft_above']].values
y = df['price'].values

# Добавим столбец единиц (свободный коэффициент)
X = np.hstack([np.ones((X.shape[0], 1)), X])

# Метод наименьших квадратов
beta_hat = np.linalg.inv(X.T @ X) @ X.T @ y

# Вывод результатов
betas_hat_table = pd.DataFrame({
    'Признак': ['intercept', 'sqft_living', 'sqft_lot', 'sqft_above'],
    'Коэффициент': beta_hat,
})

betas_hat_table

,Признак,Коэффициент
0,intercept,-41445.123606
1,sqft_living,296.151642
2,sqft_lot,-0.278184
3,sqft_above,-16.903431


Остаточная дисперсия при этом имеет вид

$$
\hat{\sigma}^2 = \frac{1}{n - p} \sum (y_i - \hat{y}_i)^2
$$

где:

- $n$ – число наблюдений
- $p$ – число параметров
- $y$ – вектор зависимой переменной
- $\hat y$ – вектор оценок зависимой переменной



In [ ]:
n, p = X.shape
y_pred = X @ beta_hat
residuals = y - y_pred

# Остаточная дисперсия
sigma_sq = np.sum(residuals**2) / (n - p)

print(f'Остаточная дисперсия: {sigma_sq:.4f}')

Остаточная дисперсия: 68179410013.8113


Для построения доверительного интервала оценок параметров используем стандартную формулу:

$$
\hat{\beta} \pm t_{1-\alpha/2} \cdot SE(\hat{\beta})
$$

где

- $t_{1-\alpha/2}$ – квантиль распреления Стьюдента
- $SE(\hat{\beta})$ – стандартная ошибка оценки $\hat{\beta}$

Приэтом стандартную ошибку можновы вычислить по следующей формуле:

$$
SE(\hat{\beta_j})=\sqrt{\left[Cov\left(\hat\beta\right)\right]_{jj}}=\sqrt{\hat{\sigma}^2\left[\left(X^TX\right)^{-1}\right]_{jj}}
$$

где

- $Cov\left(\hat\beta\right)$ – ковариационная матрица оценок

- $\left[...\right]_{jj}$ – операция взятия только диагональных элементов

In [ ]:
# Ковариационная матрица и стандартные ошибки
cov_beta = sigma_sq * np.linalg.inv(X.T @ X)
std_errors = np.sqrt(np.diag(cov_beta))

# t-квантиль
t_value = stats.t.ppf(0.975, df=n - p)

# Доверительные интервалы
conf_ints = np.vstack([beta_hat - t_value * std_errors, beta_hat + t_value * std_errors]).T

# Вывод
results_df = pd.DataFrame({
    'Признак': ['intercept', 'sqft_living', 'sqft_lot', 'sqft_above'],
    'Коэффициент': beta_hat,
    'Ст. ошибка': std_errors,
    'Доверит. интервал (нижн.)': conf_ints[:, 0],
    'Доверит. интервал (верхн.)': conf_ints[:, 1]
})

results_df

,Признак,Коэффициент,Ст. ошибка,Доверит. интервал (нижн.),Доверит. интервал (верхн.)
0,intercept,-41445.123606,4444.823515,-50157.305600,-32732.941611
1,sqft_living,296.151642,4.019923,288.272297,304.030988
2,sqft_lot,-0.278184,0.043636,-0.363713,-0.192655
3,sqft_above,-16.903431,4.467298,-25.659665,-8.147197


Рассчитаем коэффициент детерминации $R^2$:

$$
R^2 = 1-\frac{SS_{res}}{SS_{tot}} = 1 - \frac{\sum\limits_{i=1}^{n}\left(y_i-\hat{y_i}\right)^2}{\sum\limits_{i=1}^{n}\left(y_i-\overline{y}\right)^2}
$$

- $y$ – вектор зависимой переменной
- $\hat y$ – вектор оценок зависимой переменной
- $\overline y$ – среднее значение зависимой переменной

In [ ]:
SS_total = np.sum((y - y.mean())**2)
SS_res = np.sum((y - y_pred)**2)
SS_total = np.sum((y - y.mean())**2)
SS_res = np.sum((y - y_pred)**2)
R_squared = 1 - SS_res / SS_total

print(f'Коэффициент детерминации: {R_squared:.4f}')

Коэффициент детерминации: 0.4942


## Проверка гипотез

### Гипотеза 1

>Чем больше жилая площадь, тем выше цена

- $H_0$: Жилая прощадь не влияет на цену $\Rightarrow \beta_1 = 0$

- $H_1$: Жилая прощадь влияет на цену прямопропорционально $\Rightarrow \beta_1 > 0$

Для проверки гипотезы вычислим t-статистику:

$$
t = \frac{\hat\beta_1}{SE\left(\hat\beta_1\right)}
$$

In [ ]:
t_stat1 = beta_hat[1] / std_errors[1]
p_val1 = 1 - stats.t.cdf(abs(t_stat1), df=n - p)


print(f"t = {t_stat1:.3f}, p = {p_val1:.5f}")
if p_val1 < 0.05:
    print("Вывод: H0 отвергается, жилая площадь влияет на цену прямопропорционально")
else:
    print("Вывод: H0 не отвергается, жилая площадь не оказывает значимого влияния")


t = 73.671, p = 0.00000
Вывод: H0 отвергается, жилая площадь влияет на цену прямопропорционально


### Гипотеза 2

>Цена зависит от площади участка

- $H_0$: Прощадь участка не влияет на цену $\Rightarrow \beta_1 = 0$

- $H_1$: Прощадь участка влияет на цену $\Rightarrow \beta_1 \ne 0$

Для проверки гипотезы вычислим t-статистику:

$$
t = \frac{\hat\beta_2}{SE\left(\hat\beta_2\right)}
$$

In [ ]:
t_stat2 = beta_hat[2] / std_errors[2]
p_val2 = 2 * (1 - stats.t.cdf(abs(t_stat1), df=n - p))


print(f"t-статистика: {t_stat2:.3f}, p-value: {p_val2:.5f}")
if p_val2 < 0.05:
    print("Вывод: H0 отвергается, прощадь участка влияет на цену")
else:
    print("Вывод: H0 не отвергается, прощадь участка не влияет на цену ")


t-статистика: -6.375, p-value: 0.00000
Вывод: H0 отвергается, прощадь участка влияет на цену


### Гипотеза 3

- $H_0$: $\beta_1 = 0$ и $\beta_3 = 0$

- $H_1$: $\beta_1 \ne 0$ или $\beta_3 \ne 0$

Для проверки гипотезы вычислим F-статистику:

$$
F = \frac{(RSS_r - RSS_u)/q}{RSS_u/(n-p)}
$$

где

- $RSS_u$​ — полная модель (включает все признаки)

- $RSS_r$​ — ограниченная модель (не включает признаки $\beta_1$ и $\beta_3$)

- $q$ — число ограничений (в данном случае 2)

- $n$ — число наблюдений

- $p$ — число параметров в полной модели

In [ ]:
# Полная модель (уже построена)
RSS_full = residuals.T @ residuals

# Ограниченная модель (без beta1 и beta3)
X_restricted = X[:, [0, 2]]  # Только свободный член и soft_lot
beta_hat_restricted = np.linalg.inv(X_restricted.T @ X_restricted) @ X_restricted.T @ y
RSS_restricted = ((y - X_restricted @ beta_hat_restricted)**2).sum()

# F-тест
F_stat = ((RSS_restricted - RSS_full)/2) / (RSS_full/(n-p))
p_val3 = 1 - stats.f.cdf(F_stat, 2, n-p)
print(f"F-статистика: {F_stat:.3f}, p-value: {p_val3:.5f}")

if p_val3 < 0.05:
    print("Вывод: H0 отвергается, хотя бы один из признаков значимо влияет на цену.")
else:
    print("Вывод: H0 не отвергается, нет статистических оснований считать признаки значимыми.")

F-статистика: 10385.912, p-value: 0.00000
Вывод: H0 отвергается, хотя бы один из признаков значимо влияет на цену.


# Задание 2

## Цель задания

Проверить гипотезу о равенстве средних ёмкостей аккумулятора на каждом уровне фактора

- Фактор — ценовая категория (`price_range`)

- Зависимая переменная — ёмкость аккумулятора (`battery_power`)

### Формулировка гипотез:

- $ H_0 $: средние значения во всех группах равны
  
  $$
  \mu_1 = \mu_2 = \cdots = \mu_k
  $$

- $ H_1 $: хотя бы одна пара средних отличается
  
  $$
  \exists\, i,j: \mu_i \ne \mu_j
  $$


### Вычисление

1. Общая средняя:

$$
\bar{y} = \frac{1}{N} \sum_{i=1}^N y_i
$$

2. Межгрупповая сумма квадратов ($SSB$):

$$
SSB = \sum_{i=1}^{k} n_i (\bar{y}_i - \bar{y})^2
$$

3. Внутригрупповая сумма квадратов ($SSW$):

$$
SSW = \sum_{i=1}^{k} \sum_{j=1}^{n_i} (y_{ij} - \bar{y}_i)^2
$$

4. F-статистика:

$$
F = \frac{SSB / (k - 1)}{SSW / (N - k)}
$$

где:
- $ k $ — число групп,
- $ N $ — общее число наблюдений,
- $ n_i $ — размер каждой группы,
- $ \bar{y}_i $ — среднее в группе $ i $,
- $ \bar{y} $ — общее среднее.

In [ ]:
# Загрузка данных
url='https://drive.google.com/file/d/1O4rFr9xg9aFmkjx4-hl_XOc5O9q65_EW/view'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)

# Отфильтруем нужные колонки
df = df[['price_range', 'battery_power']].dropna()

# Разбиваем на группы
groups = df.groupby('price_range')['battery_power']

# Общая средняя
grand_mean = df['battery_power'].mean()
n_total = len(df)
n_groups = groups.ngroups

# Межгрупповая сумма квадратов (SSB)
SSB = sum(len(g) * (g.mean() - grand_mean)**2 for _, g in groups)

# Внутригрупповая сумма квадратов (SSW)
SSW = sum(((g - g.mean())**2).sum() for _, g in groups)

# F-статистика и p-value
F_stat = (SSB / (n_groups - 1)) / (SSW / (n_total - n_groups))
p_value = 1 - stats.f.cdf(F_stat, SSB, SSW)

print(f"F = {F_stat:.4f}, p = {p_value:.5f}")
if p_value < 0.05:
    print("Вывод: H0 отвергается — есть статистически значимые различия между группами.")
else:
    print("Вывод: H0 не отвергается — различия между группами не подтверждены.")

F = 31.5982, p = 0.00000
Вывод: H0 отвергается — есть статистически значимые различия между группами.
